**​The PRISM model, developed by Paige AI and Microsoft Research, is a multi-modal generative foundation model designed for slide-level analysis of H&E-stained histopathology images. It utilizes Virchow tile embeddings and clinical report texts for pre-training, combining these embeddings into a single slide embedding to generate text-based diagnostic reports. These reports can be used for tasks such as cancer detection, sub-typing, and biomarker identification. The model's slide encoder can also be fine-tuned for specific classification tasks, leveraging both image and text data to enhance diagnostic performance and robustness**

**Load PRISM MODEL**

In [ ]:
# from huggingface_hub import login

# login(token="")


In [ ]:
import sys
import torch
from transformers import AutoModel
from utils import VirchowTileEmbeddingExtractor, PrismProcessor
import sys
import importlib
import utils
from utils import TileExtractor

In [ ]:
scripts_path = '/content/drive/MyDrive/scripts'
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

importlib.reload(utils)

In [ ]:
sys.path.append('/content/drive/MyDrive/scripts')

model = AutoModel.from_pretrained('paige-ai/Prism', trust_remote_code=True)
model = model.to('cuda')

device = "cuda" if torch.cuda.is_available() else "cpu"
prism_processor = PrismProcessor(model_name="paige-ai/Prism", device=device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
BioGptForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/

**Initialize the VirchowTileEmbeddingExtractor**

In [ ]:
virchow_extractor = VirchowTileEmbeddingExtractor(device=device)

image_path = "outputs/img-output/image_1.png"
save_path_virchow = "outputs/embeddings-virchow/tile_virchow_1.pth"
virchow_extractor.extract_and_save_embedding(image_path, save_path_virchow)
embedding_tensor = virchow_extractor.load_embedding(save_path_virchow)
print(f"Loaded embedding shape: {embedding_tensor.shape}")


Embedding saved to: outputs/embeddings-virchow/tile_virchow_1.pth
Loaded embedding shape: torch.Size([1, 2560])


**Use generated tile embedding from WSI**

1.Tile-Embedding

In [ ]:
tile_sample_path = "outputs/embeddings-virchow/tile_virchow_1.pth"
tile_embeddings = prism_processor.load_tile_embeddings(tile_sample_path)

Loaded embedding data keys: dict_keys(['embedding'])
Tile embeddings shape: torch.Size([1, 1, 2560])


2.Virchow Tile-Embedding

In [ ]:
# tile_embeddings = embedding_tensor.unsqueeze(0).to(device)  # Adding batch dimension and moving to device
# print(f"Tile embeddings shape: {tile_embeddings.shape}")  # (batch_size, tile_seq_len, tile_embed_dim)


**Compute slide embedding | laten features**

In [ ]:
reprs = prism_processor.extract_slide_representations(tile_embeddings)

/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


Slide image embedding shape: torch.Size([1, 1280])
Slide image latents shape: torch.Size([1, 512, 1280])


**zero-shot prediction**:

Invasive lobular carcinoma (ILC) and invasive ductal carcinoma (IDC) are both types of invasive breast cancer, but they originate from different parts of the breast tissue.

IDC is the most common type of breast cancer, accounting for about 70-80% of all breast cancer cases.

ILC is less common than IDC, making up about 10-15% of breast cancers.

In [ ]:
neg_prompts = ['lobular carcinoma, invasive']
pos_prompts = ['ductal carcinoma, invasive']
scores = prism_processor.zero_shot_classification(reprs['image_embedding'], neg_prompts, pos_prompts)

Zero-shot classification scores: tensor([[0.0422, 0.9578]], device='cuda:0')


**Generate report**

In [ ]:
generated_caption = prism_processor.generate_caption(reprs['image_latents'])

Generated caption: ['</s>Diagnosis: High-grade serous carcinoma of the ovary, fallopian tube, and peritoneum. </s>']


**- logits:probabilities of the next token (word) in a sequence.**

**- text_embedding :representation of the input text (caption)**

**- image_embedding: numerical representation of the input image.**

**- image_latents:ntermediate representations generated by the Perceiver part of the PRISM model.**


In [ ]:
caption = 'Breast tissue with signs of lobular carcinoma'
output = prism_processor.make_prediction(caption, tile_embeddings)

Model output keys: dict_keys(['logits', 'text_embedding', 'image_embedding', 'image_latents', 'sim'])
